In [26]:
from keras import backend as K
K.clear_session()

In [27]:
import keras
from sklearn.utils import class_weight
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Dropout, Input
from keras.layers.merge import concatenate
from keras.models import Model
from keras.models import Sequential


In [28]:
from keras.applications import VGG16

In [29]:
from __future__ import print_function
import time
import numpy as np
import pandas as pd
from PIL import Image
import glob
import matplotlib.pyplot as plt
import os

np.random.seed(2)

In [30]:
import random
from shutil import copyfile

# Copyright 2014-2017 Bert Carremans
# Author: Bert Carremans <bertcarremans.be>
#
# License: BSD 3 clause


def img_train_test_split(img_source_dir, train_size):   
    if not (isinstance(img_source_dir, str)):
        raise AttributeError('img_source_dir must be a string')
        
    if not (isinstance(train_size, float)):
        raise AttributeError('train_size must be a float')
        
    # Set up empty folder structure if not exists
    if not os.path.exists('data'):
        os.makedirs('data')
    else:
        if not os.path.exists('data/train'):
            os.makedirs('data/train')
        if not os.path.exists('data/validation'):
            os.makedirs('data/validation')
    
    # Get the subdirectories in the main image folder
    
    subdirs = [subdir for subdir in os.listdir(img_source_dir) if os.path.isdir(os.path.join(img_source_dir, subdir))]
    
    for subdir in subdirs:
        subdir_fullpath = os.path.join(img_source_dir, subdir)
        if len(os.listdir(subdir_fullpath)) == 0:
            print(subdir_fullpath + ' is empty')
            break

        train_subdir = os.path.join('data/train', subdir)
        validation_subdir = os.path.join('data/validation', subdir)

        # Create subdirectories in train and validation folders
        if not os.path.exists(train_subdir):
            os.makedirs(train_subdir)

        if not os.path.exists(validation_subdir):
            os.makedirs(validation_subdir)

        train_counter = 0
        validation_counter = 0

        # Randomly assign an image to train or validation folder
        for filename in os.listdir(subdir_fullpath):
            if filename.endswith(".jpg") or filename.endswith(".png"): 
                fileparts = filename.split('.')

                if random.uniform(0, 1) <= train_size:
                    copyfile(os.path.join(subdir_fullpath, filename), os.path.join(train_subdir, str(train_counter) + '.' + fileparts[1]))
                    train_counter += 1
                else:
                    copyfile(os.path.join(subdir_fullpath, filename), os.path.join(validation_subdir, str(validation_counter) + '.' + fileparts[1]))
                    validation_counter += 1
                    
        print('Copied ' + str(train_counter) + ' images to data/train/' + subdir)
        print('Copied ' + str(validation_counter) + ' images to data/validation/' + subdir)

In [31]:
img_path = "/home/sogang03/school/ClimateChange_tsne/data/128px_image_square_final"
images = glob.glob(os.path.join(img_path,"*/*.jpg" ))


img_train_test_split(img_path, 0.7)

Copied 91 images to data/train/korean_rice
Copied 40 images to data/validation/korean_rice
Copied 26 images to data/train/disposable_diaper
Copied 9 images to data/validation/disposable_diaper
Copied 164 images to data/train/coffee
Copied 69 images to data/validation/coffee
Copied 46 images to data/train/china_wheat
Copied 17 images to data/validation/china_wheat
Copied 45 images to data/train/quiver_tree
Copied 29 images to data/validation/quiver_tree
Copied 190 images to data/train/aluminum_can
Copied 98 images to data/validation/aluminum_can
Copied 63 images to data/train/styrofoam
Copied 22 images to data/validation/styrofoam
Copied 99 images to data/train/newt
Copied 43 images to data/validation/newt
Copied 73 images to data/train/carpenter_bee
Copied 33 images to data/validation/carpenter_bee
Copied 47 images to data/train/long_distance_migrant_bird
Copied 24 images to data/validation/long_distance_migrant_bird
Copied 81 images to data/train/plastic_bag
Copied 29 images to data/v

In [32]:
import os
import glob

a = glob.glob('/home/sogang03/school/ClimateChange_tsne/data/train/*')
class_names = []
for i in range(len(a)):
    p = a[i][52:]
    class_names.append(p)
    
print(class_names)

['korean_rice', 'disposable_diaper', 'coffee', 'china_wheat', 'quiver_tree', 'aluminum_can', 'styrofoam', 'newt', 'carpenter_bee', 'long_distance_migrant_bird', 'plastic_bag', 'ink_cartridge', 'barley', 'spring_peeper', 'light_bulb', 'mason_bee', 'tiger', 'frog', 'adélie_penguin', 'electronic_waste', 'tin_can', 'polar_bear', 'coral', 'plastic_bottle', 'gecko', 'fishing_line', 'crocodilian', 'lizard', 'glass_bottle', 'bumblebee', 'banana', 'phytoplankton', 'snake', 'forest_bird', 'shark', 'lion', 'corn', 'snowy_plover', 'toad', 'cacao']


In [43]:
#preprocess image 

from keras.preprocessing.image import ImageDataGenerator


train_data_dir = './data/train'
validation_data_dir = './data/validation'
input_shape=(224,224,3)

trdata = ImageDataGenerator( 
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        vertical_flip=True,
        horizontal_flip=False,
        fill_mode='reflect')
traindata = trdata.flow_from_directory(directory= train_data_dir,classes=class_names, color_mode='rgb', class_mode='categorical', target_size=(224,224), batch_size=16)
vldata = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        vertical_flip=True,
        horizontal_flip=False,
        fill_mode='reflect')
valdata = vldata.flow_from_directory(directory= validation_data_dir,classes=class_names,  color_mode='rgb', class_mode='categorical', target_size=(224,224), batch_size=16)


# class_weights = class_weight.compute_class_weight('balanced', np.unique(traindata.classes), traindata.classes)



Found 5162 images belonging to 40 classes.
Found 2433 images belonging to 40 classes.


In [63]:
from keras.preprocessing.image import load_img, img_to_array

img = load_img(os.path.join(train_data_dir, 'lizard/2.jpg'))# PIL 이미지

x = img_to_array(img)  # (3, 150, 150) 크기의 NumPy 배열
x = x.reshape((1,) + x.shape)  # (1, 3, 150, 150) 크기의 NumPy 배열

# 아래 .flow() 함수는 임의 변환된 이미지를 배치 단위로 생성해서
# 지정된 `preview/` 폴더에 저장합니다.
i = 0
for batch in trdata.flow(x, batch_size=1,
                          save_to_dir='./preview', save_prefix='lizard', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # 이미지 20장을 생성하고 마칩니다

In [34]:
#Load the VGG model
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [35]:
# Freeze the layers except the last 4 layers
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False
 
# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x7f4b819ca090> False
<keras.layers.convolutional.Conv2D object at 0x7f4b819ca050> False
<keras.layers.convolutional.Conv2D object at 0x7f4b0c0dd9d0> False
<keras.layers.pooling.MaxPooling2D object at 0x7f4b819c5b50> False
<keras.layers.convolutional.Conv2D object at 0x7f4b83bdba90> False
<keras.layers.convolutional.Conv2D object at 0x7f4b83bda290> False
<keras.layers.pooling.MaxPooling2D object at 0x7f4b81826bd0> False
<keras.layers.convolutional.Conv2D object at 0x7f4b8182dbd0> False
<keras.layers.convolutional.Conv2D object at 0x7f4b81838ad0> False
<keras.layers.convolutional.Conv2D object at 0x7f4b8183f210> False
<keras.layers.pooling.MaxPooling2D object at 0x7f4b81843450> False
<keras.layers.convolutional.Conv2D object at 0x7f4b81859610> False
<keras.layers.convolutional.Conv2D object at 0x7f4b817e5510> False
<keras.layers.convolutional.Conv2D object at 0x7f4b817eb950> False
<keras.layers.pooling.MaxPooling2D object at 0x7f4b817f8b90>

In [36]:
# Create the model
model = Sequential()
 
# Add the vgg convolutional base model
model.add(vgg_conv)
 
# Add new layers
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(1024, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(40, activation='softmax'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 40)                41000     
Total params: 40,446,824
Trainable params: 32,811,560
Non-trainable params: 7,635,264
_________________________________________________________________


In [37]:
print("traindata", len(traindata))
print("valdata", len(valdata))

traindata 323
valdata 153


In [38]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 40)                41000     
Total params: 40,446,824
Trainable params: 32,811,560
Non-trainable params: 7,635,264
_________________________________________________________________


In [39]:
from keras.optimizers import Adam, RMSprop

opt = RMSprop(lr=1e-4, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [40]:
from keras.callbacks import ReduceLROnPlateau


learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [41]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("/tmp/weights.hdf5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')

hist = model.fit_generator(steps_per_epoch=200,generator=traindata, validation_data= valdata, validation_steps=10,epochs=100, callbacks=[checkpoint,early])

Epoch 1/100
200/200 [==============================] - 25s 127ms/step - loss: 3.1820 - accuracy: 0.1650 - val_loss: 2.8697 - val_accuracy: 0.2812
Epoch 2/100
  2/200 [..............................] - ETA: 16s - loss: 2.9536 - accuracy: 0.2188

/home/sogang03/.local/lib/python3.7/site-packages/keras/callbacks/callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)
/home/sogang03/.local/lib/python3.7/site-packages/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_accuracy,loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


200/200 [==============================] - 25s 126ms/step - loss: 2.6083 - accuracy: 0.2959 - val_loss: 2.8070 - val_accuracy: 0.3187
Epoch 3/100
200/200 [==============================] - 25s 123ms/step - loss: 2.2837 - accuracy: 0.3688 - val_loss: 2.3155 - val_accuracy: 0.3625
Epoch 4/100
 62/200 [========>.....................] - ETA: 17s - loss: 2.1725 - accuracy: 0.4067

KeyboardInterrupt: 

In [40]:
# Save the model
model.save('climateChange_1.h5')